In [3]:
import os
import re

def extract_info_from_entry(entry, timestamp_pattern, user_id_pattern, message_pattern):
    """
    Extracts information from a log entry using provided regular expressions.

    Args:
        entry: The log entry string
        timestamp_pattern: Regular expression to match timestamp.
        user_id_pattern: Regular expression to match user ID.
        message_pattern: Regular expression to match message.

    Returns:
        A tuple containing extracted timestamp, user ID and message (or None if not found)

    """
    timestamp = re.search(timestamp_pattern, entry). group(1) if re.search(timestamp_pattern, entry) else None
    user_id = re.search(user_id_pattern, entry).group(1) if re.search(user_id_pattern, entry) else None
    message = re.search(message_pattern, entry).group(1) if re.search(message_pattern, entry) else None
    return timestamp, user_id, message

def process_log_file(log_file, fatal_pattern, script_pattern, fatal_log_file, script_error_file, timestamp_pattern, user_id_pattern, message_pattern ):
    """
    Processes a log file, identifying and writing fatal and script errors to separate files.

    Args:
        log_file: Path to the log file.
        fatal_pattern: Regular expressions for fatal errors.
        script_pattern: Regular expressions for scritp errors.
        fatal_log_file: Path to the file for writing fatal errors.
        scritp_error_file: Path to the file for writing script errors.
        timestamp_pattern: Regular expression to match timestamp.
        user_id_pattern: Regular expression to match user ID.
        message_pattern: Regular expression to match message.
    
    """
    if not os.path.exists(log_file):
        print(f'Log file not found for {log_file}')
        return
    
    try:
        with open(log_file, 'r') as f:
            log_content = f.read()
            for entry in log_content.splitlines():   # Split by newlines
                if re.search(fatal_pattern, entry):
                    timestamp, user_id, message = extract_info_from_entry(entry, timestamp_pattern, user_id_pattern, message_pattern)
                    write_to_file(fatal_log_file, timestamp, user_id, message)
                elif re.search(script_pattern, entry):
                    timestamp, user_id, message = extract_info_from_entry(entry, timestamp_pattern, user_id_pattern, message_pattern)
                    write_to_file(script_error_file, timestamp, user_id, message)
    except FileNotFoundError:
        print(f'Error opening log file: {log_file}')

def write_to_file(file_path, timestamp, user_id, message):
    """
    Writes extracted information to a specified file.

    Args:
        file_path: Path to the file for writing.
        timestamp: Extracted timestamp from log entry.
        user_id: Extracted user ID from log entry.
        message: Extracted message from log entry.
    
    """
    try: 
        with open(file_path, 'a') as f:  # Open in append mode
            # Check if any information was extracted  (could be None)
            if timestamp and user_id and message:
                f.write(f'Timestamp: {timestamp}\nUser ID: {user_id}\nMessage: {message}\n\n')
    
    except IOError as e:
        print(f'Error writing to file {file_path}: {e}')


1. process_log_file funcion:
- It takes separate arguments for fatal_pattern and script_pattern which define the regualar expressions for identifying each type of error
- It takes separate arguments for fatal_log_file and script_error_file which specify the output file paths for fatal and script errors respectively.
2. Error Matching and Writing:
- Inside the loop that iterates through log entries:
 - If the entry matches the fatal_pattern, the extract_info_from_entry function is called to extract relevant information.
 - Then the write_to_file function is called with the fatal_log_file path, writing the extracted information to the designated file for fatal errors. 
 - Similarly, if the entry mastches the script_pattern, the information is extracted and written to the script_error_file.

By using separate patterns and file paths, the code effectively segregates fatal and scripts error logs into distinct files.
